In [1]:
import pandas as pd
import numpy as np
import random as rnd
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline

In [210]:
train_df = pd.read_csv('C:/Users/maste/PycharmProjects/Kaggle/Titanic/Dataset/train.csv')
test_df = pd.read_csv('C:/Users/maste/PycharmProjects/Kaggle/Titanic/Dataset/test.csv')

In [211]:
num_rows = train_df.shape[0]
train_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [212]:
counter_nan = train_df.isnull().sum()

In [213]:
counter_without_nan= counter_nan[counter_nan==0]

In [214]:
# Remove the column with missing elements
train_df = train_df[counter_without_nan.keys()]

In [215]:
train_df.head()

,PassengerId,Survived,Pclass,Name,Sex,SibSp,Parch,Ticket,Fare
0,1,0,3,"Braund, Mr. Owen Harris",male,1,0,A/5 21171,7.2500
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,1,0,PC 17599,71.2833
2,3,1,3,"Heikkinen, Miss. Laina",female,0,0,STON/O2. 3101282,7.9250
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,1,0,113803,53.1000
4,5,0,3,"Allen, Mr. William Henry",male,0,0,373450,8.0500


In [216]:
# Columns to normalize
cols_to_norm = ['Parch','Fare','SibSp','Pclass']


In [217]:
train_df.head()

,PassengerId,Survived,Pclass,Name,Sex,SibSp,Parch,Ticket,Fare
0,1,0,3,"Braund, Mr. Owen Harris",male,1,0,A/5 21171,7.2500
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,1,0,PC 17599,71.2833
2,3,1,3,"Heikkinen, Miss. Laina",female,0,0,STON/O2. 3101282,7.9250
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,1,0,113803,53.1000
4,5,0,3,"Allen, Mr. William Henry",male,0,0,373450,8.0500


In [218]:
train_df[cols_to_norm] = train_df[cols_to_norm].apply(lambda x: (x - x.min()) / (x.max() - x.min()))

In [219]:
train_df.head()

,PassengerId,Survived,Pclass,Name,Sex,SibSp,Parch,Ticket,Fare
0,1,0,1.0,"Braund, Mr. Owen Harris",male,0.125,0.0,A/5 21171,0.014151
1,2,1,0.0,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,0.125,0.0,PC 17599,0.139136
2,3,1,1.0,"Heikkinen, Miss. Laina",female,0.000,0.0,STON/O2. 3101282,0.015469
3,4,1,0.0,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,0.125,0.0,113803,0.103644
4,5,0,1.0,"Allen, Mr. William Henry",male,0.000,0.0,373450,0.015713


In [220]:
fare_col = tf.feature_column.numeric_column('Fare')
pclass_col = tf.feature_column.numeric_column('Pclass')
parch_col = tf.feature_column.numeric_column('Parch')
sex_col = tf.feature_column.categorical_column_with_vocabulary_list('Sex',['male','female'])
sibsp_col = tf.feature_column.numeric_column('SibSp')

In [221]:
sex_emb_col = tf.feature_column.embedding_column(sex_col, dimension=2)

In [222]:
feat_cols = [fare_col,pclass_col,parch_col,sex_emb_col,sibsp_col]

In [223]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 9 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null float64
Name           891 non-null object
Sex            891 non-null object
SibSp          891 non-null float64
Parch          891 non-null float64
Ticket         891 non-null object
Fare           891 non-null float64
dtypes: float64(4), int64(2), object(3)
memory usage: 62.7+ KB


In [240]:
x_data = train_df.drop(['Survived'],axis=1)

In [241]:
labels = train_df['Survived']

In [242]:
from sklearn.model_selection import train_test_split

In [243]:
X_train, X_test, y_train, y_test = train_test_split(x_data,labels,test_size=0.33, random_state=101)

In [244]:
input_func = tf.estimator.inputs.pandas_input_fn(x=X_train,y=y_train,batch_size=10,num_epochs=1000,shuffle=True)

In [245]:
model = tf.estimator.DNNClassifier(hidden_units=[20,20,20,20,20],feature_columns=feat_cols,n_classes=2)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': 'C:\\Users\\maste\\AppData\\Local\\Temp\\tmpyyulqsyu', '_save_checkpoints_steps': None, '_log_step_count_steps': 100, '_tf_random_seed': 1, '_keep_checkpoint_max': 5, '_session_config': None}


In [246]:
model.train(input_fn=input_func,steps=1000)

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into C:\Users\maste\AppData\Local\Temp\tmpyyulqsyu\model.ckpt.
INFO:tensorflow:loss = 6.8235, step = 1
INFO:tensorflow:global_step/sec: 338.909
INFO:tensorflow:loss = 5.58889, step = 101 (0.295 sec)
INFO:tensorflow:global_step/sec: 378.034
INFO:tensorflow:loss = 5.75265, step = 201 (0.265 sec)
INFO:tensorflow:global_step/sec: 371.816
INFO:tensorflow:loss = 1.91823, step = 301 (0.272 sec)
INFO:tensorflow:global_step/sec: 330.277
INFO:tensorflow:loss = 5.82658, step = 401 (0.300 sec)
INFO:tensorflow:global_step/sec: 336.309
INFO:tensorflow:loss = 5.46647, step = 501 (0.282 sec)
INFO:tensorflow:global_step/sec: 371.514
INFO:tensorflow:loss = 2.29123, step = 601 (0.285 sec)
INFO:tensorflow:global_step/sec: 363.779
INFO:tensorflow:loss = 4.95005, step = 701 (0.259 sec)
INFO:tensorflow:global_step/sec: 347.699
INFO:tensorflow:loss = 2.99388, step = 801 (0.303 sec)
INFO:tensorflow:global_step/sec: 336.679
INF

In [247]:
eval_input_func = tf.estimator.inputs.pandas_input_fn(
      x=X_test,
      y=y_test,
      batch_size=10,
      num_epochs=50,
      shuffle=False)

In [248]:
model.evaluate(eval_input_func)

INFO:tensorflow:Starting evaluation at 2018-02-14-16:34:40
INFO:tensorflow:Restoring parameters from C:\Users\maste\AppData\Local\Temp\tmpyyulqsyu\model.ckpt-1000
INFO:tensorflow:Finished evaluation at 2018-02-14-16:34:44
INFO:tensorflow:Saving dict for global step 1000: accuracy = 0.80678, accuracy_baseline = 0.572881, auc = 0.860101, auc_precision_recall = 0.866593, average_loss = 0.435441, global_step = 1000, label/mean = 0.427119, loss = 4.35441, prediction/mean = 0.397287


{'accuracy': 0.80677968,
 'accuracy_baseline': 0.57288134,
 'auc': 0.86010146,
 'auc_precision_recall': 0.86659253,
 'average_loss': 0.43544099,
 'global_step': 1000,
 'label/mean': 0.42711863,
 'loss': 4.3544097,
 'prediction/mean': 0.39728662}

In [249]:
test_df.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [264]:
x_test_data = test_df.drop(['PassengerId','Age','Embarked','Cabin'],axis=1)
ids = test_df['PassengerId']
x_test_data.head()
input_fn_predict = tf.estimator.inputs.pandas_input_fn(x=x_test_data,shuffle=False)

In [265]:
list(model.predict(input_fn=input_fn_predict))

INFO:tensorflow:Restoring parameters from C:\Users\maste\AppData\Local\Temp\tmpyyulqsyu\model.ckpt-1000


[{'class_ids': array([0], dtype=int64),
  'classes': array([b'0'], dtype=object),
  'logistic': array([ 0.19151786], dtype=float32),
  'logits': array([-1.44017756], dtype=float32),
  'probabilities': array([ 0.80848217,  0.19151786], dtype=float32)},
 {'class_ids': array([0], dtype=int64),
  'classes': array([b'0'], dtype=object),
  'logistic': array([ 0.11056368], dtype=float32),
  'logits': array([-2.08499622], dtype=float32),
  'probabilities': array([ 0.8894363 ,  0.11056368], dtype=float32)},
 {'class_ids': array([1], dtype=int64),
  'classes': array([b'1'], dtype=object),
  'logistic': array([ 0.73817062], dtype=float32),
  'logits': array([ 1.03648186], dtype=float32),
  'probabilities': array([ 0.26182941,  0.73817062], dtype=float32)},
 {'class_ids': array([0], dtype=int64),
  'classes': array([b'0'], dtype=object),
  'logistic': array([ 0.31798702], dtype=float32),
  'logits': array([-0.76303804], dtype=float32),
  'probabilities': array([ 0.68201298,  0.31798705], dtype=flo

In [290]:
predictions = []# np.array([])
for x in model.predict(input_fn=input_fn_predict):
    predictions.append(x['class_ids'].max())
   # np.ndarray.
    

INFO:tensorflow:Restoring parameters from C:\Users\maste\AppData\Local\Temp\tmpyyulqsyu\model.ckpt-1000


In [291]:
predictions

[0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,


In [292]:
output = pd.DataFrame({ 'PassengerId' : ids, 'Survived': predictions })
# output.to_csv('titanic-predictions.csv', index = False)
output.head()

,PassengerId,Survived
0,892,0
1,893,0
2,894,1
3,895,0
4,896,0
